In [1]:
import pandas as pd
import numpy as np
import sqlite3
import time

In [2]:
trips = pd.read_csv('../../CSVs/trips.csv', low_memory=False)
stop_times = pd.read_csv('../../CSVs/stop_times.csv', low_memory=False)
#st_update = pd.read_csv('../../CSVs/stop_time_updates.csv', usecols=['stop_sequence', 'arrival_time', 'trip_update_id'], low_memory=False)
trp_update = pd.read_csv('../../CSVs/trip_updates.csv', usecols=['oid','trip_id', 'route_id', 'timestamp', 'trip_start_time'], low_memory=False)

In [62]:
trp_update.head()

,oid,trip_id,route_id,trip_start_time,timestamp
0,1,35242107,1,NaN,2017-09-25 22:33:24.000000
1,2,35242108,1,NaN,2017-09-25 22:33:24.000000
2,3,35242109,1,NaN,2017-09-25 22:33:24.000000
3,4,35242110,1,NaN,2017-09-25 22:33:24.000000
4,5,35242111,1,NaN,2017-09-25 22:33:24.000000


In [63]:
stop_times = stop_times.loc[:, ['trip_id', 'stop_id', 'stop_sequence', 'arrival_time', 'departure_time']]
stop_times.head()

,trip_id,stop_id,stop_sequence,arrival_time,departure_time
0,Logan-22-Weekday-trip,Logan-Subway,1,08:00:00,08:00:00
1,Logan-22-Weekday-trip,Logan-RentalCarCenter,2,08:04:00,08:04:00
2,Logan-22-Weekday-trip,Logan-A,3,08:09:00,08:09:00
3,Logan-22-Weekday-trip,Logan-B,4,08:12:00,08:12:00
4,Logan-22-Weekday-trip,Logan-Subway,5,08:17:00,08:17:00


In [64]:
trips = trips.loc[:, ['trip_id', 'route_id', 'service_id', 'trip_headsign']]
trips.head()

,trip_id,route_id,service_id,trip_headsign
0,Logan-22-Weekday-trip,Logan-22,Logan-Weekday,Loop
1,Logan-22-Weekend-trip,Logan-22,Logan-Weekend,Loop
2,Logan-33-Weekday-trip,Logan-33,Logan-Weekday,Loop
3,Logan-33-Weekend-trip,Logan-33,Logan-Weekend,Loop
4,Logan-55-Weekday-trip,Logan-55,Logan-Weekday,Loop


Obtain all the first stops for all trips as well as their expected arrival time.

In [65]:
first_stops = stop_times.loc[stop_times['stop_sequence'] == 1, :]
first_stops.head()

,trip_id,stop_id,stop_sequence,arrival_time,departure_time
0,Logan-22-Weekday-trip,Logan-Subway,1,08:00:00,08:00:00
7,Logan-22-Weekend-trip,Logan-Subway,1,12:00:00,12:00:00
14,Logan-33-Weekday-trip,Logan-Subway,1,08:00:00,08:00:00
21,Logan-33-Weekend-trip,Logan-Subway,1,12:00:00,12:00:00
28,Logan-55-Weekday-trip,Logan-Subway,1,00:00:00,00:00:00


In [66]:
[trp_update['timestamp'].min(),trp_update['timestamp'].max()]

['2017-09-25 22:33:24.000000', '2017-09-26 12:33:39.000000']

Above is the time the first update and last update were obtained. Now to find trips that are running around the same time real time data was being gathered.

In [86]:
first_stops[first_stops['arrival_time'] == '09:00:00'][:25]

,trip_id,stop_id,stop_sequence,arrival_time,departure_time
79533,34663288,70276,1,09:00:00,09:00:00
194311,34686380,70276,1,09:00:00,09:00:00
230781,34706279,70001,1,09:00:00,09:00:00
234701,34706498,70001,1,09:00:00,09:00:00
254429,34723055,70001,1,09:00:00,09:00:00
272874,34849710,32003,1,09:00:00,09:00:00
274596,34849759,32001,1,09:00:00,09:00:00
284125,34849958,32004,1,09:00:00,09:00:00
287271,34850022,334,1,09:00:00,09:00:00
310600,34850726,32004,1,09:00:00,09:00:00


### Trip 34873974


In [160]:
trp_update[trp_update['trip_id'] == '34873974'].head()

,oid,trip_id,route_id,trip_start_time,timestamp
707586,707587,34873974,230,NaN,2017-09-26 12:00:55.000000
709571,709572,34873974,230,NaN,2017-09-26 12:01:49.000000
711544,711545,34873974,230,NaN,2017-09-26 12:02:29.000000
713520,713521,34873974,230,NaN,2017-09-26 12:03:11.000000
715487,715488,34873974,230,NaN,2017-09-26 12:04:30.000000


In [162]:
trp_update[trp_update['trip_id'] == '34873974'][-1:]

,oid,trip_id,route_id,trip_start_time,timestamp
786091,786092,34873974,230,NaN,2017-09-26 12:33:39.000000


All of these updates were retreived at hour 12, but the expected arrival time for the first stop is at hour 9.

In [75]:
stop_times[stop_times['trip_id'] == '34873974'].head()

,trip_id,stop_id,stop_sequence,arrival_time,departure_time
396399,34873974,32003,1,09:00:00,09:00:00
396400,34873974,3326,2,09:01:00,09:01:00
396401,34873974,3327,3,09:03:00,09:03:00
396402,34873974,3328,4,09:04:00,09:04:00
396403,34873974,3330,5,09:05:00,09:05:00


Above is the expected arrival times for trip 34873974

In [76]:
stop_times[stop_times['trip_id'] == '34873974'].shape

(74, 5)

There are 74 stops

In [3]:
# connect to the sqlite database for querying data from stop time updates
conn = sqlite3.connect('../../gtfs.db')

In [4]:
first_update = pd.read_sql_query('select * from stop_time_updates where trip_update_id = 707587', conn)
first_update.head()

,oid,stop_sequence,stop_id,arrival_delay,arrival_time,arrival_uncertainty,departure_delay,departure_time,departure_uncertainty,schedule_relationship,trip_update_id
0,16546640,1,32003,0,1506430860,0,0,1506430860,0,SCHEDULED,707587
1,16546641,2,3326,0,1506431130,0,0,1506431130,0,SCHEDULED,707587
2,16546642,3,3327,0,1506431255,0,0,1506431255,0,SCHEDULED,707587
3,16546643,4,3328,0,1506431287,0,0,1506431287,0,SCHEDULED,707587
4,16546644,5,3330,0,1506431399,0,0,1506431399,0,SCHEDULED,707587


We can see that for the first update we the predicted arrival times for all the stops in trip 34873974. Lets look at the predicted arrival times from the second update (trip_update_id = 709572)

In [78]:
second_update = pd.read_sql_query('select * from stop_time_updates where trip_update_id = 709572', conn)
second_update.head()

,oid,stop_sequence,stop_id,arrival_delay,arrival_time,arrival_uncertainty,departure_delay,departure_time,departure_uncertainty,schedule_relationship,trip_update_id
0,16591893,1,32003,0,1506430880,0,0,1506430880,0,SCHEDULED,709572
1,16591894,2,3326,0,1506431150,0,0,1506431150,0,SCHEDULED,709572
2,16591895,3,3327,0,1506431276,0,0,1506431276,0,SCHEDULED,709572
3,16591896,4,3328,0,1506431307,0,0,1506431307,0,SCHEDULED,709572
4,16591897,5,3330,0,1506431420,0,0,1506431420,0,SCHEDULED,709572


On the second update we still have the predicted arrival times for all the stops. Lets compare the predicted arrival time from the first and second update. Their epoch times differ, but lets get a more human readable format.

In [79]:
first_update['arrival_time'].apply(lambda x : time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(x))).head()

0    2017-09-26 08:01:00
1    2017-09-26 08:05:30
2    2017-09-26 08:07:35
3    2017-09-26 08:08:07
4    2017-09-26 08:09:59
Name: arrival_time, dtype: object

In [80]:
second_update['arrival_time'].apply(lambda x : time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(x))).head()

0    2017-09-26 08:01:20
1    2017-09-26 08:05:50
2    2017-09-26 08:07:56
3    2017-09-26 08:08:27
4    2017-09-26 08:10:20
Name: arrival_time, dtype: object

The predicted arrival times for the first stop differ by just 20 seconds. Looks like the vehicle is predicted to arrive early according to the first two updates

In [164]:
final_update = pd.read_sql_query('select * from stop_time_updates where trip_update_id = 786092', conn)
final_update.head()

,oid,stop_sequence,stop_id,arrival_delay,arrival_time,arrival_uncertainty,departure_delay,departure_time,departure_uncertainty,schedule_relationship,trip_update_id
0,18301848,1,32003,0,1506430969,0,0,1506430969,0,SCHEDULED,786092
1,18301849,2,3326,0,1506431239,0,0,1506431239,0,SCHEDULED,786092
2,18301850,3,3327,0,1506431364,0,0,1506431364,0,SCHEDULED,786092
3,18301851,4,3328,0,1506431396,0,0,1506431396,0,SCHEDULED,786092
4,18301852,5,3330,0,1506431509,0,0,1506431509,0,SCHEDULED,786092


In [84]:
final_update['arrival_time'].apply(lambda x : time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(x))).head()

0    2017-09-26 08:02:49
1    2017-09-26 08:07:19
2    2017-09-26 08:09:24
3    2017-09-26 08:09:56
4    2017-09-26 08:11:49
Name: arrival_time, dtype: object

This is the final update in the trip updates table. The final update is still reporting a predicted time for the first stop sequence. 

### Trip 35242107
Lets look at another trip. Trip 35242107

In [75]:
tr_up = trp_update[trp_update['trip_id'] == '35242107']
tr_up

,oid,trip_id,route_id,trip_start_time,timestamp
0,1,35242107,1,NaN,2017-09-25 22:33:24.000000
1471,1472,35242107,1,NaN,2017-09-25 22:34:06.000000
2940,2941,35242107,1,NaN,2017-09-25 22:34:44.000000
4407,4408,35242107,1,NaN,2017-09-25 22:35:37.000000
5872,5873,35242107,1,NaN,2017-09-25 22:36:11.000000
7334,7335,35242107,1,NaN,2017-09-25 22:36:55.000000
8790,8791,35242107,1,NaN,2017-09-25 22:37:42.000000
10230,10231,35242107,1,NaN,2017-09-25 22:38:30.000000
11666,11667,35242107,1,NaN,2017-09-25 22:39:13.000000
13089,13090,35242107,1,NaN,2017-09-25 22:40:20.000000


In [90]:
stop_times[stop_times['trip_id'] == '35242107'].head()

,trip_id,stop_id,stop_sequence,arrival_time,departure_time
1619730,35242107,110,1,18:55:00,18:55:00
1619731,35242107,2168,2,18:56:00,18:56:00
1619732,35242107,2166,3,18:57:00,18:57:00
1619733,35242107,2167,4,18:58:00,18:58:00
1619734,35242107,66,5,18:58:00,18:58:00


Here are some of the expected arrival times. What follows are a series of predicted arrival times from different trip updates.

In [135]:
update = pd.read_sql_query('select * from stop_time_updates where trip_update_id = 1', conn)
update.head()

,oid,stop_sequence,stop_id,arrival_delay,arrival_time,arrival_uncertainty,departure_delay,departure_time,departure_uncertainty,schedule_relationship,trip_update_id
0,1,1,110,0,1506380100,0,0,1506380100,0,SCHEDULED,1
1,2,2,2168,0,1506380204,0,0,1506380204,0,SCHEDULED,1
2,3,3,2166,0,1506380276,0,0,1506380276,0,SCHEDULED,1
3,4,4,2167,0,1506380331,0,0,1506380331,0,SCHEDULED,1
4,5,5,66,0,1506380399,0,0,1506380399,0,SCHEDULED,1


In [136]:
update['arrival_time'].apply(lambda x : time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(x))).head()

0    2017-09-25 17:55:00
1    2017-09-25 17:56:44
2    2017-09-25 17:57:56
3    2017-09-25 17:58:51
4    2017-09-25 17:59:59
Name: arrival_time, dtype: object

For the first update we see that the predicted arrival times are ahead of schedule by approximately an hour.

In [137]:
update = pd.read_sql_query('select * from stop_time_updates where trip_update_id = 37456', conn)
update.head()

,oid,stop_sequence,stop_id,arrival_delay,arrival_time,arrival_uncertainty,departure_delay,departure_time,departure_uncertainty,schedule_relationship,trip_update_id
0,879187,1,110,0,1506380388,0,0,1506380388,0,SCHEDULED,37456
1,879188,2,2168,0,1506380467,0,0,1506380467,0,SCHEDULED,37456
2,879189,3,2166,0,1506380525,0,0,1506380525,0,SCHEDULED,37456
3,879190,4,2167,0,1506380566,0,0,1506380566,0,SCHEDULED,37456
4,879191,5,66,0,1506380615,0,0,1506380615,0,SCHEDULED,37456


In [138]:
update['arrival_time'].apply(lambda x : time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(x))).head()

0    2017-09-25 17:59:48
1    2017-09-25 18:01:07
2    2017-09-25 18:02:05
3    2017-09-25 18:02:46
4    2017-09-25 18:03:35
Name: arrival_time, dtype: object

In [139]:
update = pd.read_sql_query('select * from stop_time_updates where trip_update_id = 40015', conn)
update.head()

,oid,stop_sequence,stop_id,arrival_delay,arrival_time,arrival_uncertainty,departure_delay,departure_time,departure_uncertainty,schedule_relationship,trip_update_id
0,939593,1,110,0,1506380427,0,0,1506380427,0,SCHEDULED,40015
1,939594,2,2168,0,1506380505,0,0,1506380505,0,SCHEDULED,40015
2,939595,3,2166,0,1506380563,0,0,1506380563,0,SCHEDULED,40015
3,939596,4,2167,0,1506380604,0,0,1506380604,0,SCHEDULED,40015
4,939597,5,66,0,1506380653,0,0,1506380653,0,SCHEDULED,40015


In [140]:
update['arrival_time'].apply(lambda x : time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(x))).head()

0    2017-09-25 18:00:27
1    2017-09-25 18:01:45
2    2017-09-25 18:02:43
3    2017-09-25 18:03:24
4    2017-09-25 18:04:13
Name: arrival_time, dtype: object

In [156]:
update = pd.read_sql_query('select * from stop_time_updates where trip_update_id = 55936', conn)
update.head()

,oid,stop_sequence,stop_id,arrival_delay,arrival_time,arrival_uncertainty,departure_delay,departure_time,departure_uncertainty,schedule_relationship,trip_update_id
0,1310246,1,110,0,1506380689,0,0,1506380689,0,SCHEDULED,55936
1,1310247,2,2168,0,1506380767,0,0,1506380767,0,SCHEDULED,55936
2,1310248,3,2166,0,1506380825,0,0,1506380825,0,SCHEDULED,55936
3,1310249,4,2167,0,1506380866,0,0,1506380866,0,SCHEDULED,55936
4,1310250,5,66,0,1506380915,0,0,1506380915,0,SCHEDULED,55936


In [157]:
update['arrival_time'].apply(lambda x : time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(x))).head()

0    2017-09-25 18:04:49
1    2017-09-25 18:06:07
2    2017-09-25 18:07:05
3    2017-09-25 18:07:46
4    2017-09-25 18:08:35
Name: arrival_time, dtype: object

In [165]:
update = pd.read_sql_query('select * from stop_time_updates where trip_update_id = 57104', conn)
update.head()

,oid,stop_sequence,stop_id,arrival_delay,arrival_time,arrival_uncertainty,departure_delay,departure_time,departure_uncertainty,schedule_relationship,trip_update_id
0,1337164,2,2168,0,1506380767,0,0,1506380767,0,SCHEDULED,57104
1,1337165,3,2166,0,1506380825,0,0,1506380825,0,SCHEDULED,57104
2,1337166,4,2167,0,1506380866,0,0,1506380866,0,SCHEDULED,57104
3,1337167,5,66,0,1506380915,0,0,1506380915,0,SCHEDULED,57104
4,1337168,6,67,0,1506380980,0,0,1506380980,0,SCHEDULED,57104


In [166]:
update['arrival_time'].apply(lambda x : time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(x))).head()

0    2017-09-25 18:06:07
1    2017-09-25 18:07:05
2    2017-09-25 18:07:46
3    2017-09-25 18:08:35
4    2017-09-25 18:09:40
Name: arrival_time, dtype: object

Trip update 57104 does not provide a predicted arrival time for the first stop. The last predicted time was from update 55936 with a prediction of 18:04:49. Now the second stop is predicted to arrive at 18:06:07.

In [ ]:
stp_upd_ids = trp_update[trp_update['trip_id'] == '35242107']['oid'].values
stp_upd_ids = [str(x) for x in stp_upd_ids]

The above code gets all the trip update ids that correspond to trip 35242107.
The ids are currently integers, but we will convert them to strings due to complications with pandas parameter binding for SQL

In [100]:
place_holders = ','.join('?' * len(stp_upd_ids))
sql = 'SELECT * FROM stop_time_updates WHERE trip_update_id IN ({})'.format(place_holders)
updates_df = pd.read_sql_query(sql, conn, params=stp_upd_ids)
updates_df.head()

,oid,stop_sequence,stop_id,arrival_delay,arrival_time,arrival_uncertainty,departure_delay,departure_time,departure_uncertainty,schedule_relationship,trip_update_id
0,1,1,110,0,1506380100,0,0,1506380100,0,SCHEDULED,1
1,2,2,2168,0,1506380204,0,0,1506380204,0,SCHEDULED,1
2,3,3,2166,0,1506380276,0,0,1506380276,0,SCHEDULED,1
3,4,4,2167,0,1506380331,0,0,1506380331,0,SCHEDULED,1
4,5,5,66,0,1506380399,0,0,1506380399,0,SCHEDULED,1


We can see the first few rows of the updates_df dataframe. This dataframe holds all predicted arrival times associated with trip_updated_ids from stp_upd_ids.

In [101]:
updates_df = pd.merge(updates_df, tr_up.loc[:, ['oid', 'trip_id', 'timestamp']], left_on=['trip_update_id'], right_on=['oid'])

In [102]:
updates_df.head()

,oid_x,stop_sequence,stop_id,arrival_delay,arrival_time,arrival_uncertainty,departure_delay,departure_time,departure_uncertainty,schedule_relationship,trip_update_id,oid_y,trip_id,timestamp
0,1,1,110,0,1506380100,0,0,1506380100,0,SCHEDULED,1,1,35242107,2017-09-25 22:33:24.000000
1,2,2,2168,0,1506380204,0,0,1506380204,0,SCHEDULED,1,1,35242107,2017-09-25 22:33:24.000000
2,3,3,2166,0,1506380276,0,0,1506380276,0,SCHEDULED,1,1,35242107,2017-09-25 22:33:24.000000
3,4,4,2167,0,1506380331,0,0,1506380331,0,SCHEDULED,1,1,35242107,2017-09-25 22:33:24.000000
4,5,5,66,0,1506380399,0,0,1506380399,0,SCHEDULED,1,1,35242107,2017-09-25 22:33:24.000000


The updates_df dataframe has been merged with tr_up. tr_up holds all the trip_updates_ids corresponding to trip 35242107 and a timestamp that indicates when this update was retreived

In [108]:
idx = updates_df.groupby('stop_sequence')['timestamp'].transform(max) == updates_df['timestamp']
updates_df[idx].head()

,oid_x,stop_sequence,stop_id,arrival_delay,arrival_time,arrival_uncertainty,departure_delay,departure_time,departure_uncertainty,schedule_relationship,trip_update_id,oid_y,trip_id,timestamp
1175,1310246,1,110,0,1506380689,0,0,1506380689,0,SCHEDULED,55936,55936,35242107,2017-09-25 23:04:58.000000
1311,1444897,2,2168,0,1506380877,0,0,1506380877,0,SCHEDULED,61725,61725,35242107,2017-09-25 23:08:14.000000
1415,1549626,3,2166,0,1506381044,0,0,1506381044,0,SCHEDULED,66246,66246,35242107,2017-09-25 23:10:53.000000
1441,1575673,4,2167,0,1506381086,0,0,1506381086,0,SCHEDULED,67376,67376,35242107,2017-09-25 23:11:35.000000
1490,1627062,5,66,0,1506381165,0,0,1506381165,0,SCHEDULED,69602,69602,35242107,2017-09-25 23:12:53.000000


updates_df is grouped using the stop_sequence column. This is so we can find the latest update (timestamp) per stop_sequence value. Once the maximum timestamps are found, boolean indexing is used in order to display the last predicted arrival time obtained from the real time feed for each stop. We can see below that we obtained the same last predicted arrival time for the first stop using this method as we did doing a manual search. 

In [110]:
time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(1506380689))

'2017-09-25 18:04:49'